# PDF Utils: extending current functionalities for PDF

Add dependency to `pdfminer.six`: use `pip install unpackai[PDF]` to install the dependencies.

In [ ]:
# default_exp pdf

In [ ]:
# export
import re
from pathlib import Path
from typing import List, Union

from pdfminer.high_level import extract_text
from unpackai import utils
from unpackai import nlp

PathStr = Union[Path, str]

In [ ]:
# export
class TextualPDF(nlp.Textual):
    """Extend Textual for PDF"""

    @classmethod
    def from_url_pdf(
        cls, url: str, password: str = "", page_numbers: List[int] = None, cleanup=True
    ):
        """Create a Textual object from a PDF URL with specific options

        Args:
            url: url of PDF
            password: password, if the PDF is protected
            page_numbers: list of pages to extract (first page = 0)
            cleanup: remove messy characters and line returns (default=True)
        """
        return cls.from_path_pdf(
            utils.download(url),
            password=password,
            page_numbers=page_numbers,
            cleanup=cleanup,
        )

    @classmethod
    def from_path_pdf(
        cls,
        pdf_file: PathStr,
        password: str = "",
        page_numbers: List[int] = None,
        cleanup=True,
    ):
        """Create a Textual object from a PDF

        Args:
            pdf_file: path of PDF
            password: password, if the PDF is protected
            page_numbers: list of pages to extract (first page = 0)
            cleanup: remove messy characters and line returns (default=True)
        """
        txt = extract_text(pdf_file, password=password, page_numbers=page_numbers)
        if cleanup:
            txt = re.sub(r"[\r\n]{2,}", "<line_break>", txt)
            txt = re.sub(r"- *[\n\r]", "", txt)
            txt = txt.replace("\n", " ").replace("<line_break>", "\n\n")

        return cls(txt, Path(pdf_file))

    @classmethod
    def from_path(cls, path: PathStr):
        """Create a Textual object from a path, including PDF"""
        path = Path(path)
        if path.suffix.lower() == ".pdf":
            return cls.from_path_pdf(path)
        else:
            return super().from_path(path)


## How to use it?

```python

from unpackai.pdf import TextualPDF

textual = TextualPDF.from_url("http://islamicblessings.com/upload/A-Thousand-And-One-Nights-1.pdf")
# OR...
textual = TextualPDF.from_path("C:/my_doc.pdf")
# OR ... if there is a password or you want to extract specific pages...
textual = TextualPDF.from_url_pdf("https://my_company.com/my_protected_doc.pdf", password="P@ssW0rd")
textual = TextualPDF.from_path_pdf("C:/my_doc.pdf", page_numbers=range(10))
```

In [ ]:
t = TextualPDF.from_url("http://islamicblessings.com/upload/A-Thousand-And-One-Nights-1.pdf")
t

Custom TB Handler failed, unregistering


---------------------------------------------------------------------------
gaierror                                  Traceback (most recent call last)
C:\Python39\lib\site-packages\urllib3\connection.py in _new_conn(self)
    173         try:
--> 174             conn = connection.create_connection(
    175                 (self._dns_host, self.port), self.timeout, **extra_kw

C:\Python39\lib\site-packages\urllib3\util\connection.py in create_connection(address, timeout, source_address, socket_options)
     72 
---> 73     for res in socket.getaddrinfo(host, port, family, socket.SOCK_STREAM):
     74         af, socktype, proto, canonname, sa = res

C:\Python39\lib\socket.py in getaddrinfo(host, port, family, type, proto, flags)
    952     addrlist = []
--> 953     for res in _socket.getaddrinfo(host, port, family, type, proto, flags):
    954         af, socktype, proto, canonname, sa = res

gaierror: [Errno 11001] getaddrinfo failed

During handling of the above exception, another e

# Tests

In [ ]:
# hide
# To be able to run the tests in the Notebook
from pathlib import Path
import ipytest
import sys

ipytest.autoconfig()

root_dir = Path("..").resolve()
sys.path.append(str(root_dir / "test"))

In [ ]:
# exportest
# For Test Cases (might have duplicate import because it will be in a dedicated file)
import re
from pathlib import Path
from typing import List

import pytest
from test_common.utils_4_tests import DATA_DIR, compare_strings
from test_utils import GITHUB_TEST_DATA_URL, check_connection_github
from unpackai import nlp


In [ ]:
# hide

## --- Checking the compare function

exp_lines = """
Hello, my name is
John

How are you
today
?
"""

obt_lines = """
Hello, my name is
Jeff

How are you
?
"""

print(compare_strings(obt_lines, exp_lines))

--- Expected [string #1]
+++ Obtained [string #2]
@@ -1,6 +1,7 @@
 
 Hello, my name is
-Jeff
+John
 
 How are you
+today
 ?



In [ ]:
# exportest
GITHUB_TEST_PDF = f"{GITHUB_TEST_DATA_URL}/Deep%20learning.pdf"
LOCAL_TEST_PDF = DATA_DIR / "Deep learning.pdf"
LOCAL_TEST_TXT = DATA_DIR / "Deep learning.txt"

LOCAL_TEST_PDF_SMALL = DATA_DIR / "to_download.pdf"
LOCAL_TEST_TXT_SMALL = DATA_DIR / "to_download.txt"


@pytest.fixture(scope="session")
def local_textual():
    return TextualPDF.from_path(LOCAL_TEST_PDF)


@pytest.fixture(scope="session")
def test_pdf_as_txt():
    return Path(LOCAL_TEST_TXT).read_text(encoding="utf-8")


def cleanup_spaces(text: str) -> str:
    return re.sub(r"[\r\n\t\s]+", " ", text, flags=re.S)


class TestTextualPDF:
    def test_from_path(self, local_textual):
        """Test extract Textual of PDF from local file"""
        t = local_textual
        assert "Deep learning" in t.text, f"Text parsed:\n{t.text}"

    def test_from_path_not_pdf(self):
        """Test extract Textual of non-PDF file"""
        textual = TextualPDF.from_path(LOCAL_TEST_TXT)
        assert textual.text == nlp.Textual.from_path(LOCAL_TEST_TXT).text

    def test_from_path_pdf(self, local_textual):
        """Test extract Textual of PDF from local path using from_path_pdf"""
        textual = TextualPDF.from_path_pdf(LOCAL_TEST_PDF)
        assert textual.text == local_textual.text

    @pytest.mark.parametrize(
        "pdf, txt, encoding",
        [
            (LOCAL_TEST_PDF_SMALL, LOCAL_TEST_TXT_SMALL, None),
            (LOCAL_TEST_PDF, LOCAL_TEST_TXT, "utf-8"),
        ],
        ids=["small", "utf-8"],
    )
    def test_from_path_pdf_no_cleanup(self, pdf: Path, txt: Path, encoding):
        """Test extract Textual from PDF without cleanup"""
        content = cleanup_spaces(txt.read_text(encoding=encoding))

        textual = TextualPDF.from_path_pdf(pdf, cleanup=False)
        textual_txt = cleanup_spaces(textual.text)
        assert textual_txt == content, compare_strings(content, textual_txt)

    @pytest.mark.parametrize("pages", [[0], [0, 1], [0, 2], range(1, 4)])
    def test_from_path_pdf_pages(self, pages):
        """Test extract Textual from PDF with specific pages"""
        text = TextualPDF.from_path_pdf(LOCAL_TEST_PDF, page_numbers=pages).text
        for p in pages:
            assert f"Page {p+1}" in text, f"Page {p+1} not found in Textual: {text}"
        pages_extracted = re.findall(r"Page (\d+)", text)
        extra_pages = set(pages_extracted) - set(str(p + 1) for p in pages)
        assert extra_pages == set(), f"Extra pages extracted: {extra_pages}"

    def test_from_url(self, check_connection_github, local_textual):
        """Test extract Textual of PDF from URL"""
        textual = TextualPDF.from_url(GITHUB_TEST_PDF)
        assert textual.text == local_textual.text, f"URL text: {textual.text}"

    def test_from_url_pdf(self, check_connection_github, local_textual):
        """Test extract Textual of PDF from URL using from_url_pdf"""
        textual = TextualPDF.from_url_pdf(GITHUB_TEST_PDF)
        assert textual.text == local_textual.text, f"URL text: {textual.text}"

    # TODO: Add Tests with password


In [ ]:
# hide
ipytest.run()

.........                                                                                    [100%]##vso[results.publish type=JUnit;runTitle='Pytest results';]e:\AnsysDev\_perso_repo\unpackai\nbs\test-output.xml
##vso[task.logissue type=warning;]Coverage XML was not created, skipping upload.

------------ generated xml file: e:\AnsysDev\_perso_repo\unpackai\nbs\test-output.xml -------------
9 passed in 3.73s
